In [ ]:
# !pip install detecto

In [ ]:
import glob
import numpy as np
import io
import os


import torch
import torchvision
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter
# from torchvision.models.detection import 

import skimage
from PIL import Image
from detecto.core import Model
from detecto.visualize import detect_live, detect_video, plot_prediction_grid, show_labeled_image
from detecto.core import DataLoader, Dataset
from detecto.utils import read_image, xml_to_csv, normalize_transform

import matplotlib.pyplot as plt

In [ ]:
# model = Model()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# !du -sh /Users/haridas/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth

In [ ]:
# detect_live(model)
train_dir = "/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05/train"
test_dir = "/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05/test"


# train_dir = "/home/haridas/projects/mystique/data/train_and_test-2020-05-31/train"
# test_dir = "/home/haridas/projects/mystique/data/train_and_test-2020-05-31/test"

In [ ]:
# plot_prediction_grid
train_labels = xml_to_csv(
    train_dir,
    f"{train_dir}/../train_label.csv"
)
val_labels = xml_to_csv(
    test_dir,
    f"{test_dir}/../test_label.csv"
)

classes = train_labels['class'].unique().tolist()

In [ ]:
train_labels["class"].value_counts()

In [ ]:
train_labels["class"].value_counts()

In [ ]:
# train_labels.filename.unique()
# !pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
normalize_transform()

In [ ]:
# Image reader and pre-processing pipeline.
transformer = T.Compose([
    T.ToPILImage(),
    lambda image: image.convert("RGB"),
    T.ToTensor(),
    normalize_transform()
])

In [ ]:
# Pytorch dataset for train and validation.
dataset = Dataset(
    f"{train_dir}/../train_label.csv",
    image_folder=train_dir,
    transform=transformer
)

val_dataset = Dataset(
    f"{test_dir}/../test_label.csv",
    image_folder=test_dir,
    transform=transformer
)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=2)
val_dataloader = DataLoader(val_dataset, batch_size=2)

In [ ]:
# ims, lbs = dataset[100]
# show_labeled_image(ims, lbs["boxes"])

In [ ]:
# !pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# torch.cuda("cuda:0")
# torch.cuda.is_available()
# f"{test_dir}/../train_label.csv"

## Train models in GPU

In [ ]:
torch.cuda.device_count()

In [ ]:
new_model = Model(classes)

In [ ]:
# dataset._csv.describe()
# _model = torch.nn.DataParallel(new_model)

In [ ]:
new_model.fit(dataset, val_dataset=val_dataset, verbose=True)

In [ ]:
tb_writer = SummaryWriter("Second")
new_model = CustomModel(classes, log_writer=tb_writer)

In [ ]:
new_model.fit(train_dataloader, val_dataset=val_dataloader, verbose=True, epochs=20)

In [ ]:
# model = Model(classes=["test", 'asdf'])
# model.predict([img])
# new_model.name
#save("/home/haridas/projects/pic2card/model/pth_models/")

In [ ]:
# torch.cuda.memory_stats()
# model.predict([img])

In [ ]:
# %debug

# Load saved model and test

In [ ]:
model_path_20epoch = "/home/haridas/projects/pic2card/model/pth_models/faster-rcnn-2020-05-31-1590914103.pth",
model_path_25epoch = "/home/haridas/projects/pic2card/model/pth_models/faster-rcnn-2020-05-31-1590943544-epochs_25.pth"

model_path_35epoch = "/home/haridas/projects/pic2card-models/pytorch/faster-rcnn-2020-06-17-1592424185-epochs_35.pth"

# model_path_10epoch = "/home/haridas/projects/pic2card/model/pth_models/faster-rcnn-2020-05-31-1590928573-epochs_10.pth"

In [ ]:
classes

In [ ]:
# Load the saved model
model = Model.load("pic2card_model.pth", classes=classes)

In [ ]:
model = Model.load(
    model_path_25epoch,
    classes=classes
)

In [ ]:
img = transformer(read_image(f"{train_dir}/1.png"))

In [ ]:
# show_labeled_image(
# T.ToPILImage(read_image(f"{train_dir}/1.png"))
# img.shape
im = read_image(f"{test_dir}/104.png")

In [ ]:
type(im_tfs)

In [ ]:
im = read_image(f"{test_dir}/104.png")
im_tfs = transformer(im)
labels, boxes, scores = model.predict([im_tfs])[0]
# show_labeled_image(im, boxes, labels)

In [ ]:
# list(zip(labels, scores))
# model.predict([im_tfs])[0]
# np.array(classes)

In [ ]:
labels, boxes, scores = model.predict(img)
torchvision.transforms.ToPILImage()(img)
# show_labeled_image(img, boxes, labels)
# img.shape
show_labeled_image(img, boxes, labels)

In [ ]:
# new_model._get_raw_predictions([img])
# Send the image to gpu.
# img = img.to(new_model._device)
# new_model._model([img])

In [ ]:
# model.predict([img])
# new_model.predict([img])

In [ ]:
# !ls ../../mystique/data/train_and_test/train
# new_model._device
# %debug

# Tensorboard 

In [ ]:
from torch.utils.tensorboard import SummaryWriter, RecordWriter

In [ ]:
writer = SummaryWriter("testing")

In [ ]:
writer.add_image("images", torchvision.utils.make_grid([img]), 0)

In [ ]:
# writer.add_scalar?

In [ ]:
# writer.add_image?
writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)